# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.235179941925                   -0.50    8.0
  2   -7.250421397233       -1.82       -1.40    1.0
  3   -7.251320096781       -3.05       -2.18    3.0
  4   -7.251297337985   +   -4.64       -2.31    3.0
  5   -7.251331750366       -4.46       -2.74    2.0
  6   -7.251337956816       -5.21       -3.15    2.0
  7   -7.251338790951       -6.08       -4.15    1.0
  8   -7.251338797349       -8.19       -4.53    4.0
  9   -7.251338798623       -8.89       -5.16    3.0
 10   -7.251338798703      -10.10       -5.84    2.0
 11   -7.251338798704      -11.96       -6.43    2.0
 12   -7.251338798705      -12.90       -6.82    3.0
 13   -7.251338798705      -13.51       -7.58    2.0
 14   -7.251338798705      -14.75       -8.06    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.03922020409277574
[ Info: Arnoldi iteration step 2: normres = 0.5429151809476288
[ Info: Arnoldi iteration step 3: normres = 0.8941554859018614
[ Info: Arnoldi iteration step 4: normres = 0.4132563908628417
[ Info: Arnoldi iteration step 5: normres = 0.3947730994467661
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (5.91e-02, 7.14e-02, 3.21e-01, 2.10e-01, 1.62e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3731084049216086
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.95e-02, 1.91e-01, 2.84e-01, 1.05e-01, 4.49e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07677366706418545
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (7.61e-04, 1.80e-02, 1.33e-02, 3.58e-02, 5.80e-02)
[ Info: Arnoldi iteration step 8: normres = 0.09154984754735458
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (2.93e-0